In [1]:
tf_bin_path = "../data/bin/linux/"
model_path = "../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"
frozen_graph_v1 = "../data/models/birdnet-frozen-v1.pb"
opt_graph_v1 = "../data/models/birdnet-opt-v1.pb"
onnx_model_v1 = "../data/models/birdnet-onnx-v1.onnx"

! gsutil rsync -r gs://birdclef-2023/data/bin/linux/ {tf_bin_path}
! mkdir -p ../data/models
! source ../venv/bin/activate

Building synchronization state...
Starting synchronization...


In [13]:
import torch
from onnx2torch import convert
import onnx

Something is not working with the onnx2torch. Dug into it and the onnx model graph doesn't have any nodes in it? I am not sure if the format is not what the conversion package is expecting for some reason. See the following code.

In [3]:
onnx_model_path = "../data/models/birdnet-onnx-v1.onnx"
onnx_model = onnx.load(onnx_model_path)
torch_model_2 = convert(onnx_model_path)

self output values: {}


KeyError: 'unknown_242:0'

Below we see the output and input nodes, but no nodes specified in between.

In [4]:
onnx_model.graph

name: "tf2onnx"
initializer {
  dims: 3337
  data_type: 1
  name: "unknown_244:0"
  raw_data: "\267\317\234\274\001c\021\276\327\346\312\2750]\274=\321\367\251\275\026\3244\276?\326\251\275\"\376\204<\307\300A\276\206\016\246=\370\2353;xH\312\275\357\301z\275\003\237o\276\216\331)>B(\214\276\320\223\200\275\246$\376\275\252>\243\2762\241\002>\017\371\253\275R\237\314\275\334b\230\275|\231{\275\217|W\276\313\220b\276\300\365\347\2743\203\204>\324\357\265\274\213\210+\276\254\240\312\276\036\225\037\276\330\253\024=\003f\335\274W\'\203\275~\263\261\276\231\302\"\275Xd}\275X\220\n\276\321\311\322\273~\004\021\276u\276(\275*8x<\\\177\205\275\214\005\'\276\301\303\020=\312\367\026\271\0329\274\275\276V\364\275M\026\210=\002|,\276\223j\206\276\204\211\235=\235M]=\307W\005\274v`M\276*\270\027\276O\212{\276\341Ra\276\317\340\307\274\277\202\326\275[\r\267\276~>\036>\002\353\026\276\'\210\r=\365\002\305=\342\341_\276/f6\275\347D\024\275;\277\\<\004\303\232\273\225\025\007\275\035;\222\275j]\232

In [5]:
nodes = onnx_model.graph.node
print(nodes)

[]


Reproduce Error from converter.py

In [7]:
from onnx2torch.utils.safe_shape_inference import safe_shape_inference
from onnx.onnx_ml_pb2 import ModelProto
from torch import fx
from torch import nn

from onnx2torch.node_converters import get_converter
from onnx2torch.onnx_graph import OnnxGraph
from onnx2torch.onnx_graph import ValueType


def _remove_initializers_from_input(model: ModelProto) -> ModelProto:
    graph_inputs = model.graph.input
    # print('graph_inputs:',graph_inputs)
    graph_inputs_mapping = {one_input.name: one_input for one_input in graph_inputs}
    # print('graph_inputs_mapping:', graph_inputs_mapping)

    for initializer in model.graph.initializer:
        if initializer.name in graph_inputs_mapping:
            print("initilizer.name")
            graph_inputs.remove(graph_inputs_mapping[initializer.name])

    return model


class InitializersContainer(nn.Module):
    """Module for storing initializers in torch fx graph."""

    def add_initializer(
        self, name: str, initializer: torch.Tensor
    ) -> None:  # pylint: disable=missing-docstring
        self.register_buffer(name, initializer)

    def forward(self, *args, **kwargs):  # pylint: disable=missing-function-docstring
        raise RuntimeError('Got unexpected "forward" on constant container')


onnx_model_test = safe_shape_inference(onnx_model)
onnx_model_new = _remove_initializers_from_input(onnx_model_test)
onnx_graph = OnnxGraph(onnx_model_new.graph)
torch_graph = fx.Graph()
opset_import = {
    opsetid_proto.domain: opsetid_proto.version
    for opsetid_proto in onnx_model.opset_import
}

torch_initializers = InitializersContainer()
torch_modules = nn.Module()
torch_modules.add_module("initializers", torch_initializers)
torch_nodes = {}

save_input_names = False
for input_value, name in enumerate(onnx_graph.input_values, 1):
    # print('input_value:', input_value)
    # print('name:',name)
    if save_input_names:
        if not name.isidentifier():
            raise ValueError(
                f'Input name "{name}" cannot be used as name of placeholder in fx.GraphModule.'
            )

        placeholder_name = name
    else:
        placeholder_name = f"input_{input_value}"

    torch_nodes[name] = torch_graph.placeholder(name=placeholder_name)

self output values: {}


In [8]:
graph_inputs = onnx_model.graph.input
graph_inputs_mapping = {one_input.name: one_input for one_input in graph_inputs}
output_values = tuple(value_info.name for value_info in onnx_model.graph.output)
unique_names = []
counters = {}

for node in onnx_model.graph.node:
    name = f"{node.domain}_{node.op_type}".lstrip("_")
    print("name:", name)
    name_counter = counters.setdefault(name, 0)
    counters[name] += 1
    unique_names.append(f"{name}_{name_counter}")

# onnx_graph = OnnxGraph(onnx_model_test.graph)
# pylint: disable=no-member
torch_graph = fx.Graph()

torch_initializers = InitializersContainer()
torch_modules = nn.Module()
torch_modules.add_module("initializers", torch_initializers)
torch_nodes = {}
attach_onnx_mapping = False
for name, onnx_node in onnx_graph.nodes.items():
    print("name:", name)
    print("onnx_node:", node)
    version = opset_import[onnx_node.domain]
    converter = get_converter(
        domain=onnx_node.domain,
        operation_type=onnx_node.operation_type,
        version=version,
    )

    torch_module, onnx_mapping = converter(onnx_node, onnx_graph)
    if attach_onnx_mapping:
        setattr(torch_module, "onnx_mapping", onnx_mapping)

    torch_modules.add_module(name, torch_module)

    args = []
    for value_name in onnx_mapping.inputs:
        value_type = onnx_graph.value_type(value_name)
        if value_type == ValueType.GRAPH_INPUT:
            args.append(torch_nodes[value_name])

        elif value_type == ValueType.NODE_OUTPUT:
            onnx_input_node, _ = onnx_graph.value_as_node_output(value_name)
            torch_input_node = torch_nodes[onnx_input_node.unique_name]

            # Get only one needed output of torch_input_node by index
            if len(onnx_input_node.output_values) > 1:
                index = onnx_input_node.output_values.index(value_name)
                torch_input_node = torch_graph.call_function(
                    getitem, args=(torch_input_node, index)
                )
                torch_nodes[name + "_split_output"] = torch_input_node
            args.append(torch_input_node)

        elif value_type == ValueType.GRAPH_INITIALIZER:
            # The name of pytorch buffer must not contain '.'(dot)
            len_torch_initializers = sum(1 for _ in torch_initializers.buffers())
            torch_buffer_name = f"onnx_initializer_{len_torch_initializers}"
            if value_name not in torch_nodes:
                torch_initializers.add_initializer(
                    torch_buffer_name,
                    onnx_graph.initializers[value_name].to_torch(),
                )
                torch_nodes[torch_buffer_name] = torch_graph.get_attr(
                    f"initializers.{torch_buffer_name}"
                )
            args.append(torch_nodes[torch_buffer_name])

        elif value_type == ValueType.EMPTY:
            args.append(None)

        else:
            raise RuntimeError(f"Got unexpected input value type ({value_type})")
    kwargs = {}
    if None in args:
        first_skipped_arg = args.index(None)
        forward_args = tuple(inspect.signature(torch_module.forward).parameters.keys())
        forward_args = forward_args[first_skipped_arg : len(args)]
        args, kwargs_values = args[:first_skipped_arg], args[first_skipped_arg:]
        kwargs.update(
            {
                name: value
                for name, value in zip(forward_args, kwargs_values)
                if value is not None
            }
        )

    torch_nodes[name] = torch_graph.call_module(
        module_name=name, args=tuple(args), kwargs=kwargs
    )


# THIS IS WHERE THE ERROR IS
onnx_output_nodes = [
    onnx_graph.value_as_node_output(value_name)[0]
    for value_name in onnx_graph.output_values
]
# Delete duplicates and save order

# onnx_output_nodes = list(OrderedDict.fromkeys(onnx_output_nodes))

KeyError: 'unknown_242:0'

Now let's dig into onnx_graph.py:

In [9]:
from onnx2torch.onnx_node import OnnxNode
from onnx2torch.onnx_tensor import OnnxTensor
from typing import Mapping
from typing import Tuple

from onnx.onnx_ml_pb2 import GraphProto
from onnx.onnx_ml_pb2 import ValueInfoProto
from collections import OrderedDict


proto = onnx_model_new.graph
input_values = tuple(value_info.name for value_info in proto.input)
output_values = tuple(value_info.name for value_info in proto.output)

unique_names = []
counters = {}
for node in proto.node:
    # print("is there a node?", node)
    name = f"{node.domain}_{node.op_type}".lstrip("_")
    print("name:", name)
    name_counter = counters.setdefault(name, 0)
    counters[name] += 1
    unique_names.append(f"{name}_{name_counter}")

Nothing in unique names:

In [10]:
print(unique_names)

[]


In [12]:
onnx_graph_nodes = OrderedDict(
    (name, OnnxNode(node, unique_name=name))
    for name, node in zip(unique_names, proto.node)
)
initializers = {
    initializer.name: OnnxTensor(initializer) for initializer in proto.initializer
}
node_output_values = {
    output_name: (node, i)
    for node in onnx_graph_nodes.values()
    for i, output_name in enumerate(node.output_values)
}
print(node_output_values)

{}
